<h3>Load the data, taking the first sample</h3>

In [4]:
import pandas as pd
import numpy as np
df = pd.DataFrame.from_csv('mzMATCHoutput.txt',sep='\t');
mass = df.index.tolist()
rt = df['RT'].tolist()
intensity = df['T0R1L'].tolist()
rid = df['relation.id'].tolist()

<h3>Group fragments if they are within 1 ppm of each other</h3>

In [38]:
unique_masses = []
mass_id = []
tol = 1
for m in mass:
    # check for previous
    previous_pos = [i for i,a in enumerate(unique_masses) if (abs(m-a)/m)*1e6 < tol]
    if len(previous_pos) == 0:
        # it's a new one
        unique_masses.append(m)
        mass_id.append(len(unique_masses)-1)
    else:
        # it's an old one
        mass_id.append(previous_pos[0])

<h3>Create the data matrix and then trim to get rid of rare fragments, and dodgy data items</h3>

In [121]:
dmat = np.zeros((len(unique_masses),max(rid)+1),np.float)
for i,m in enumerate(mass):
    dmat[mass_id[i],rid[i]] = intensity[i]
    
    
    
min_met = 2
r,c = dmat.shape
remove = []
col_names = np.array(range(max(rid)+1))
row_names = np.array(unique_masses)
for i in range(r):
    s = np.where(dmat[i,:]>0)[0]
    if len(s)<min_met:
        remove.append(i)

remove = np.array(remove)
row_names = np.delete(row_names,remove)
dmat = np.delete(dmat,remove,axis=0)
min_frag = 3
r,c = dmat.shape
remove = []
for i in range(c):
    s = np.where(dmat[:,i]>0)[0]
    if len(s)<min_frag:
        remove.append(i)
remove = np.array(remove)

col_names = np.delete(col_names,remove)
dmat = np.delete(dmat,remove,axis=1)

# Remove fragments that appear nowhere
remove = []
for i in range(r):
    s = np.where(dmat[i,:]>0)[0]
    if len(s) == 0:
        remove.append(i)

dmat = np.delete(dmat,remove,axis=0)
row_names = np.delete(row_names,remove)

print dmat.shape,row_names.shape,col_names.shape

(1268, 332) (1268,) (332,)


<h3>Turn into integer array with biggest peak in each spectra at 100</h3>

In [103]:
dmat_int = np.zeros(dmat.shape,np.int)
r,c = dmat.shape
for i in range(c):
    ma = dmat[:,i].max()
    dmat_int[:,i] = 100*dmat[:,i]/ma
    

<h3>Make into a Pandas Structure to export</h3>

In [129]:
d = pd.DataFrame(dmat_int,index=row_names,columns = col_names)
d.to_csv('gcms_for_lda.csv')

,0,2,4,6,8,10,11,12,14,15,...,1249,1263,1269,1315,1333,1372,1434,1570,1670,1811
298.064220,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
300.060738,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
224.045374,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
282.032989,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
313.087696,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209.993316,6,0,0,0,0,0,0,100,0,0,...,0,0,0,0,0,0,0,0,0,0
189.993766,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
191.973062,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150.026639,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179.946395,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
